<div class="alert alert-block alert-info">
This script merges all eegh files for a session and splits them back according to session timestamps. The Jupyer Notebook works well for a single
session/troubleshooting, and the Python script is better for multiple sessions.
</div>

In [1]:
import numpy as np
import os
import csv
import json
import argparse

## Variables
These variables may change:

In [2]:
animal_name='JC315'
date='20240401'

basedir = "/mnt/adata11/"
mbasedir="/adata_pool/merged/"+animal_name+'-'+date+'/'

These variables do not change:

In [3]:
sample_rate_res_old=24000

sample_rate_whl=39.0625
sample_rate_res=20000

sample_rate_eegh=5000

downsampled_res=sample_rate_res/sample_rate_res_old

## Import session metadata:

In [4]:
session_metadata = {}
with open('session_metadata.csv', mode='r', encoding='utf-8') as file:
    reader = csv.DictReader(file, delimiter=";")    
    for line in reader:
        session_id = line.pop('session_id')
        session_metadata[session_id] = line

In [5]:
basename = animal_name+'-'+date
print('Metadata for',basename)
session_metadata[basename]

Metadata for JC315-20240401


{'num_tetr': '32',
 'last_pfc_left': '8',
 'last_pfc_right': '16',
 'session_names': "['presleep','training1','intersleep','training2','postsleep']",
 'session_idx': '[[1],[2,3,4],[5],[6,7],[8]]',
 'reward_arms': '7,2',
 'rewards': 'C,S'}

In [6]:
num_tetrodes = int(session_metadata[basename]['num_tetr'])

session_idx = json.loads(session_metadata[basename]['session_idx'])

session_names_str = session_metadata[basename]['session_names']
session_names_str = session_names_str.replace("'",'"') # the single quotes aren't being read in JSON, but I can't use " in the csv because it's a str delimiter
session_names = json.loads(session_names_str)

reward_arms_str = session_metadata[basename]['reward_arms']
reward_arms = np.array(list(map(int, reward_arms_str.split(','))))

## Merge and split eegh files and export reward arm files

In [7]:
def find_files_with_eegh(directory):
    eegh_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if 'eegh' in file:
                eegh_files.append(os.path.join(root, file))
                
    return eegh_files

In [8]:
# basedir='D:\github/test/'
# mbasedir="D:\github/test\merged"+animal_name+'-'+date+'/'
if not os.path.isdir(mbasedir):
    os.makedirs(mbasedir)
directory_path_eegh = basedir+"eeg/"+animal_name+'/'+date+'/'

In [9]:
# find all eegh files in a session
eegh_files = sorted(find_files_with_eegh(directory_path_eegh)) # sort the filenames numerically

# initialize a merged eegh file and add the first file to it
eegh_merged=np.fromfile(eegh_files[0], dtype=np.int16)
print('Reshaping',eegh_files[0],'from length',eegh_merged.shape)
# reshape the first file (1D to 2D array) so that each row corresponds to one tetrode
eegh_merged= eegh_merged.reshape(int(len(eegh_merged)/num_tetrodes),num_tetrodes)
print('New eegh_merged shape:',eegh_merged.shape)

# iterate over the other files and add them to the large merged file
for eegh_file_i in range(1,len(eegh_files)):
    eegh_t=np.fromfile(eegh_files[eegh_file_i], dtype=np.int16)
    print('Reshaping',eegh_files[eegh_file_i], 'from length',eegh_t.shape)
    eegh_t= eegh_t.reshape(int(len(eegh_t)/num_tetrodes),num_tetrodes)
    print('New eegh_merged shape:',eegh_t.shape)
    eegh_merged=np.append(eegh_merged,eegh_t,axis=0)

print('Final eegh_merged shape:',eegh_merged.shape)
    
# calculate the total length (num timestamps) of the merged eegh file
length_eegh_merged=eegh_merged.shape[0]

Reshaping /mnt/adata11/eeg/JC315/20240401/JC315-20240401_01.eegh from length (576000000,)
New eegh_merged shape: (18000000, 32)
Reshaping /mnt/adata11/eeg/JC315/20240401/JC315-20240401_02.eegh from length (285966336,)
New eegh_merged shape: (8936448, 32)
Reshaping /mnt/adata11/eeg/JC315/20240401/JC315-20240401_03.eegh from length (412471296,)
New eegh_merged shape: (12889728, 32)
Reshaping /mnt/adata11/eeg/JC315/20240401/JC315-20240401_04.eegh from length (633131008,)
New eegh_merged shape: (19785344, 32)
Reshaping /mnt/adata11/eeg/JC315/20240401/JC315-20240401_05.eegh from length (576000000,)
New eegh_merged shape: (18000000, 32)
Reshaping /mnt/adata11/eeg/JC315/20240401/JC315-20240401_06.eegh from length (483405824,)
New eegh_merged shape: (15106432, 32)
Reshaping /mnt/adata11/eeg/JC315/20240401/JC315-20240401_07.eegh from length (583487488,)
New eegh_merged shape: (18233984, 32)
Reshaping /mnt/adata11/eeg/JC315/20240401/JC315-20240401_08.eegh from length (576000000,)
New eegh_merged

In [11]:
# load the session timestamps and downsample them
session_timestamps=np.loadtxt(basedir+"processing/"+animal_name+'/'+date+'/'+'session_shifts.txt')

session_timestamps=np.append([0],session_timestamps) # start the first timestamp at 0
session_timestamps_down=session_timestamps*downsampled_res
print('Resampled session timestamps:',session_timestamps)

for session_idx_i in range(len(session_idx)):
    # generate reward arms files for the training session files
    if session_names[session_idx_i]=='training1' or session_names[session_idx_i]=='training2':
        np.savetxt(mbasedir+animal_name+'-'+date+'_'+session_names[session_idx_i]+'.reward_arms', reward_arms, fmt='%i',newline=" ")
    
    # cut the eegh files according to the session timestamps and write them to the merged folder
    start_cut=session_idx[session_idx_i][0]-1
    end_cut=session_idx[session_idx_i][-1]
    start_eegh=int(session_timestamps_down[start_cut]/sample_rate_res*sample_rate_eegh)
    end_eegh=int(session_timestamps_down[end_cut]/sample_rate_res*sample_rate_eegh)

    eegh_temp=eegh_merged[start_eegh:end_eegh,:]

    eegh_temp.tofile(mbasedir+animal_name+'-'+date+'_'+session_names[session_idx_i]+'.eegh')

print('Done merging eeg files for',basename)

Resampled session timestamps: [0.00000000e+00 8.64001470e+07 1.29295194e+08 1.91166141e+08
 2.86136088e+08 3.72536235e+08 4.45047132e+08 5.32570629e+08
 6.18970776e+08]
Done merging eeg files for JC315-20240401
